# Cargar Dependencias

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns; sns.set(style="darkgrid")
import matplotlib.pyplot as plt

# Conseguir Dataframe de trabajo

In [ ]:
dataset = pd.read_csv("base_dataset.csv", index_col="patient_id")
dataset.head()

In [ ]:
interest_cols = ["birth_date", "start_date", "start_night", "end_night", "measure_date_time"]
for col in interest_cols:
    dataset[col] = pd.to_datetime(dataset[col])

In [ ]:
dataset["age"] = (dataset["start_date"] - dataset["birth_date"]).apply(lambda x: x.total_seconds() / (365.25 * 24 * 60 * 60))

In [ ]:
dataset.head()

In [ ]:
max_cols = [
    'treatment',
    'birth_date',
    'start_date',
    'start_night',
    'end_night',
    'gender',
    'height',
    'weight',
    'age',
    'smoking',
    'excercise_frecuency',
    'beer',
    'wine',
    'alcohol',
    'strong_licor',
]
mean_cols = [
    'sistolic',
    'diastolic',
    'heart_rate',
    'mean_arterial_pressure',
    'pulse_pressure',
    'stroke_volume',
    'cardiac_output',
    'cardiac_index',
    'systemic_vascular_resistance',
    'pulse_wave_velocity'        
]

In [ ]:
grouped_dataset_1 = dataset[max_cols].groupby("patient_id").max()
grouped_dataset_2 = dataset[mean_cols].groupby("patient_id").mean()

In [ ]:
grouped_dataset = pd.merge(
    grouped_dataset_1, 
    grouped_dataset_2, 
    how="inner", 
    left_index=True, 
    right_index=True
)



In [ ]:
grouped_dataset.head()

In [ ]:
grouped_dataset.to_csv('working_dataset.csv')

# Crear random Dataframe para compartir
Como estos datos tienen informacion de los pacientes no se pueden compartir libremente. Para desarrollar el proyecto integrador, se genero una base de datos con la misma matriz de covarianza de la original, pero con datos aleatorizados bajo una distribucion gaussiana. Este permite el desarrollo de codigos por parte de los integrantes del grupo que despues seran usados en la base de datos final.

In [ ]:
from sklearn.preprocessing import StandardScaler

In [ ]:
categorical_columns = [
    'treatment',
    'gender',
    'smoking',
    'excercise_frecuency',
    'beer',
    'wine',
    'alcohol',
    'strong_licor'
]

numerical_columns = [
    'height',
    'weight',
    'age',
    'sistolic',
    'diastolic',
    'heart_rate',
    'mean_arterial_pressure',
    'pulse_pressure',
    'stroke_volume',
    'cardiac_output',
    'cardiac_index',
    'systemic_vascular_resistance',
    'pulse_wave_velocity'
]

## Generacion de grupos.
Este codigo lo que hace es generar datasets aleatorios en los que:
- 1. filtrar por todas las posibles combinaciones de las variables cualitativas
- 2. estandarizar este subgrupo
- 3. calcular la matriz de covarianza de este subgrupo
- 4. generar un array aleatorio de la misma cantidad de datos que el subgrupo 
- 5. transformar inversamente la estandarizacion
- 6. repetir los pasos 2 al 5 por cada subgrupo

In [ ]:
random_datasets = []
for a in grouped_dataset[categorical_columns[0]].unique():
    for b in grouped_dataset[categorical_columns[1]].unique()[:-1]:
        for c in grouped_dataset[categorical_columns[2]].unique():
            for d in grouped_dataset[categorical_columns[3]].unique():
                for e in grouped_dataset[categorical_columns[4]].unique():
                    for f in grouped_dataset[categorical_columns[5]].unique():
                        for g in grouped_dataset[categorical_columns[6]].unique():
                            for h in grouped_dataset[categorical_columns[7]].unique():
                                random_dataset = grouped_dataset[
                                    (grouped_dataset[categorical_columns[0]] == a) &
                                    (grouped_dataset[categorical_columns[1]] == b) &
                                    (grouped_dataset[categorical_columns[2]] == c) &
                                    (grouped_dataset[categorical_columns[3]] == d) &
                                    (grouped_dataset[categorical_columns[4]] == e) &
                                    (grouped_dataset[categorical_columns[5]] == f) &
                                    (grouped_dataset[categorical_columns[6]] == g) &
                                    (grouped_dataset[categorical_columns[7]] == h)
                                ].copy()
                                std_random = StandardScaler()
                                try:
                                    std_random.fit(random_dataset[numerical_columns])
                                except ValueError:
                                    continue
                                random_ = std_random.transform(random_dataset[numerical_columns])
                                try:
                                    random_array = np.random.multivariate_normal(random_.mean(axis=0), np.cov(random_,rowvar=False), random_.shape[0])
                                except ValueError:
                                    continue
                                random_data = pd.DataFrame(std_random.inverse_transform(random_array), columns=numerical_columns)
                                for col, val in list(zip(categorical_columns, [a, b, c, d, e, f, g, h])):
                                    random_data[col] = val
                                random_datasets.append(random_data)
                           
                            


## Concatenar los datasets aleatorios

In [ ]:
random_df = pd.concat(
    random_datasets, 
    ignore_index=True
)
print(random_df.shape)
random_df["index"] = np.random.choice(random_df.shape[0],random_df.shape[0], False)
random_df.set_index("index",inplace=True)
random_df.sort_index(inplace=True)
random_df.head()

In [ ]:
random_df.to_csv("random_dataframe.csv")